# MIPT DB HW 2

#### Libraries

In [34]:
# Импортируем модуль для подключения и работы с PostgreSQL:
import psycopg2 as pg

# Импортируем модуль для работы с данными:
import pandas as pd

#### PostgreSQL Connection

In [35]:
try:
    # Установливаем соединение с PostgreSQL:
    conn = pg.connect(dbname='postgres', host='localhost', user='postgres', password='Seven8ins', port=5432)
    # Создаем курсор для выполнения SQL-запросов:
    cursor = conn.cursor()
    # Выполняем простой SQL-запрос SELECT 1 для проверки соединения:
    cursor.execute("SELECT 1")
    # Если все прошло успешно, выводим сообщение об успешном соединении:
    print("Connection successful")


# Если произошла ошибка при подключении или выполнении запроса, выводим сообщение об ошибке:
except Exception as e:
    print("An error occurred:", e)

Connection successful


#### Data

In [10]:
# Указываем путь к файлам:
path_transaction = 'transaction.csv'
path_customer = 'customer.csv'

# Загружаем данные из CSV-файла в DataFrame pandas:
transaction = pd.read_csv(path_transaction, delimiter=';')
customer = pd.read_csv(path_customer, delimiter=';')

In [11]:
# Заменяем запятые на точки и преобразуем значения в тип float:
for col in ['standard_cost', 'list_price']:
    transaction[col] = transaction[col].str.replace(',', '.').astype(float)

# Преобразуем столбец 'transaction_date' в формат datetime:
transaction['transaction_date'] = pd.to_datetime(transaction['transaction_date'], format='%d.%m.%Y')

# Определяем функцию для замены значений в столбце 'gender':
def replace_gender(gender):
    if gender == 'U':
        return 'Unknown'
    elif gender == 'F':
        return 'Female'
    elif gender == 'M':
        return 'Male'
    elif gender == 'Femal':
        return 'Female'
    else:
        return gender

# Применяем функцию replace_gender к столбцу 'gender':
customer['gender'] = customer['gender'].apply(replace_gender)

# Преобразуем столбец 'DOB' в формат datetime:
customer['DOB'] = pd.to_datetime(customer['DOB'])

In [12]:
# Отображаем три случайные строки из DataFrame 'transaction':
display(transaction.sample(3))

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
13870,13871,55,1244,2017-12-11,True,Approved,Trek Bicycles,Road,medium,large,1894.19,598.76
11211,11212,74,763,2017-08-16,False,Approved,WeareA2B,Standard,medium,medium,1228.07,400.91
9279,9280,40,1017,2017-01-10,False,Approved,Trek Bicycles,Road,medium,large,1894.19,598.76


In [13]:
# Отображаем три случайные строки из DataFrame 'customer':
display(customer.sample(3))

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
2826,2827,Shelli,Wilkennson,Female,1973-07-15,NaN,Financial Services,High Net Worth,N,No,82 Vermont Street,2021,NSW,Australia,12
1957,1958,Janina,Keme,Female,1985-09-01,Professor,NaN,Mass Customer,N,No,29 Hansons Crossing,2318,NSW,Australia,4
1477,1478,Fulton,Blayney,Male,1992-12-05,VP Quality Control,Financial Services,High Net Worth,N,No,7231 Pepper Wood Point,2428,NSW,Australia,8


#### Import Data to PostgreSQL

In [59]:
# Откатываем текущую транзакцию:
try:
    conn.rollback()
except:
    pass

In [14]:
# Сохраняем DataFrame 'transaction' в CSV-файл без индексов:
transaction.to_csv('transaction.csv', index=False)

# Выполняем SQL-запрос для создания таблицы 'transaction', если она еще не существует:
cursor.execute("""
               CREATE TABLE IF NOT EXISTS transaction (
               transaction_id int4,
               product_id int4,
               customer_id int4, 
               transaction_date date, 
               online_order varchar(30),
               order_status varchar(30),
               brand varchar(30),
               product_line varchar(30),
               product_class varchar(30),
               product_size varchar(30),
               list_price float4, 
               standard_cost float4
               """)

# Открываем CSV-файл и копируем его содержимое в таблицу 'transaction' в базе данных:
with open('transaction.csv', 'r') as f:
    # Пропускаем заголовок:
    next(f)  
    cursor.copy_expert("COPY transaction FROM STDIN CSV HEADER", f)

# Подтверждаем транзакцию, чтобы изменения были сохранены в базе данных:
conn.commit()

In [15]:
# Сохраняем DataFrame 'customer' в CSV-файл без индексов:
customer.to_csv('customer.csv', index=False)

# Выполняем SQL-запрос для создания таблицы 'customer', если она еще не существует:
cursor.execute("""
               CREATE TABLE IF NOT EXISTS customer (
               customer_id int4,
               first_name varchar(50),
               last_name varchar(50),
               gender varchar(30),
               DOB date, 
               job_title varchar(50),
               job_industry_category varchar(50),
               wealth_segment varchar(50),
               deceased_indicator varchar(50),	
               owns_car varchar(50),	
               address varchar(50),
               postcode	int4,
               state varchar(50),
               country varchar(50),
               property_valuation int4
               )
               """)

# Открываем CSV-файл и копируем его содержимое в таблицу 'customer' в базе данных:
with open('customer.csv', 'r') as f:
    next(f)
    cursor.copy_expert("COPY customer FROM STDIN CSV HEADER", f)

# Подтверждаем транзакцию, чтобы изменения были сохранены в базе данных:
conn.commit()

In [36]:
# Выполняем SQL-запрос, который извлекает первые три строки из таблицы 'transaction':
cursor.execute("SELECT * FROM transaction LIMIT 3")

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Выводим каждую строку:
for row in rows:
    print(row)

# Подтверждаем транзакцию:
conn.commit()

(2, 3, 3120, datetime.date(2017, 5, 21), 'True', 'Approved', 'Trek Bicycles', 'Standard', 'medium', 'large', 2091.47, 388.92)
(3, 37, 402, datetime.date(2017, 10, 16), 'False', 'Approved', 'OHM Cycles', 'Standard', 'low', 'medium', 1793.43, 248.82)
(4, 88, 3135, datetime.date(2017, 8, 31), 'False', 'Approved', 'Norco Bicycles', 'Standard', 'medium', 'medium', 1198.46, 381.1)


In [17]:
# Выполняем SQL-запрос, который извлекает первые три строки из таблицы 'customer':
cursor.execute("SELECT * FROM customer LIMIT 3")

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Выводим каждую строку:
for row in rows:
    print(row)

# Подтверждаем транзакцию:
conn.commit()

(2, 'Eli', 'Bockman', 'Male', datetime.date(1980, 12, 16), 'Administrative Officer', 'Financial Services', 'Mass Customer', 'N', 'Yes', '6 Meadow Vale Court', 2153, 'New South Wales', 'Australia', 10)
(3, 'Arlin', 'Dearle', 'Male', datetime.date(1954, 1, 20), 'Recruiting Manager', 'Property', 'Mass Customer', 'N', 'Yes', '0 Holy Cross Court', 4211, 'QLD', 'Australia', 9)
(4, 'Talbot', None, 'Male', datetime.date(1961, 10, 3), None, 'IT', 'Mass Customer', 'N', 'No', '17979 Del Mar Point', 2448, 'New South Wales', 'Australia', 4)


#### SQL queries

In [68]:
# Откатываем текущую транзакцию:
try:
    conn.rollback()
except:
    pass

* (1 балл) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [18]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT brand 
               FROM transaction 
               WHERE standard_cost::int > 1500
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,brand
0,OHM Cycles
1,Trek Bicycles
2,Solex
3,Giant Bicycles


* (1 балл) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

In [19]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT * 
               FROM transaction 
               WHERE order_status = 'Approved' AND transaction_date BETWEEN '2017-04-01' AND '2017-04-09'
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,17,79,2426,2017-04-03,False,Approved,Norco Bicycles,Standard,medium,medium,1555.58,818.01
1,19,54,2268,2017-04-06,True,Approved,WeareA2B,Standard,medium,medium,1292.84,13.44
2,23,37,2001,2017-04-08,True,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,83,0,3398,2017-04-01,True,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07
4,89,0,2682,2017-04-04,True,Approved,OHM Cycles,Road,high,large,12.01,7.21
...,...,...,...,...,...,...,...,...,...,...,...,...
2119,19655,0,336,2017-04-09,True,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30
2120,19853,7,3072,2017-04-02,False,Approved,Trek Bicycles,Road,low,medium,980.37,234.43
2121,19899,57,325,2017-04-06,False,Approved,WeareA2B,Touring,medium,large,1890.39,260.14
2122,19968,0,2751,2017-04-06,False,Approved,WeareA2B,Standard,medium,medium,60.34,45.26


* (1 балл) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [20]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT job_title 
               FROM customer
               WHERE (job_industry_category = 'IT' 
                    OR job_industry_category = 'Financial Services') 
                    AND job_title LIKE 'Senior%'
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,job_title
0,Senior Sales Associate
1,Senior Financial Analyst
2,Senior Cost Accountant
3,Senior Editor
4,Senior Quality Engineer
5,Senior Developer


* (1 балл) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

In [43]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT brand 
               FROM transaction 
               WHERE brand IS NOT NULL 
                    AND customer_id IN (
                        SELECT customer_id 
                        FROM customer 
                        WHERE job_industry_category = 'Financial Services')
""")

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,brand
0,OHM Cycles
1,Trek Bicycles
2,WeareA2B
3,Solex
4,Norco Bicycles
5,Giant Bicycles


* (1 балл) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [60]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT c.customer_id, c.first_name, c.last_name
               FROM transaction t
               JOIN customer c ON t.customer_id = c.customer_id
               WHERE t.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles') AND t.online_order = 'True'
               LIMIT 10;
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,customer_id,first_name,last_name
0,742,Dexter,Robelin
1,20,Basile,Firth
2,3123,Tina,Riggulsford
3,1894,Patten,Laytham
4,3426,Ron,Dilon
5,3491,Leanna,Cromb
6,1179,Kerry,Pashenkov
7,2201,Trisha,Basset
8,3095,Joachim,None
9,1521,Pernell,Duffett


* (1 балл) Вывести всех клиентов, у которых нет транзакций.

In [62]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT c.customer_id, c.first_name, c.last_name
               FROM customer c
               LEFT JOIN transaction t ON c.customer_id = t.customer_id
               WHERE t.transaction_id IS NULL
""")

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,customer_id,first_name,last_name
0,852,Andie,Bonney
1,869,Addia,Abels
2,1373,Shaylynn,Epsley
3,2074,Roslyn,Rawdall
4,2660,Hunt,Scollard
...,...,...,...
502,3996,Rosalia,Halgarth
503,3997,Blanch,Nisuis
504,3998,Sarene,Woolley
505,3999,Patrizius,None


* (2 балла) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [63]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT c.customer_id, c.first_name, c.last_name
               FROM customer c
               JOIN transaction t ON c.customer_id = t.customer_id
               WHERE c.job_industry_category = 'IT'
                    AND t.standard_cost = (
                        SELECT MAX(standard_cost)
                        FROM transaction)
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,customer_id,first_name,last_name
0,34,Jephthah,Bachmann
1,893,Gibby,Fearnley
2,975,Goldarina,Rzehorz
3,1672,Sharla,Creebo
4,1773,Nickolas,Guittet
5,1918,Devin,Sandeson
6,2913,Padraic,Bonnar
7,3151,Thorn,Choffin
8,3473,Sanderson,Alloway


* (2 балла) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [70]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT DISTINCT c.customer_id, c.first_name, c.last_name
               FROM customer c
               JOIN transaction t ON c.customer_id = t.customer_id
               WHERE c.job_industry_category IN ('IT', 'Health')
                    AND t.transaction_date 
                        BETWEEN '2017-07-07' AND '2017-07-17' 
                        AND t.order_status = 'Approved'
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,customer_id,first_name,last_name
0,2341,Caterina,Scedall
1,1489,Mead,Wollard
2,612,Grange,Skillington
3,2000,Kain,Rene
4,3365,Karlens,Chaffyn
...,...,...,...
110,712,Norine,Antonik
111,22,Deeanne,Durtnell
112,892,Conroy,Healy
113,2881,Ezra,Steffan


In [27]:
# Закрываем курсор
cursor.close()

# Закрываем соединение с базой данных
conn.close()